In [ ]:
from component.CargaInicialComponent import CargaInicialComponent
from model.constants.ParametrosConstantes import ParametrosConstantes
from service.CargaInicialService import CargaInicialService
from service.MunicipioService import MunicipioService
from service.UnidadeFederativaService import UnidadeFederativaService
from service.client.IBGEClientService import IBGEClientService
from util.BancoDadosUtil import BancoDadosUtil
from util.LoggerUtil import LoggerUtil
from util.OSUtil import OSUtil
from util.WarningUtil import WarningUtil

In [ ]:
# Configurações iniciais

WarningUtil.ignorar_warning(categoria=DeprecationWarning)
WarningUtil.ignorar_warning(categoria=UserWarning)

OSUtil.criar_diretorio(diretorio=ParametrosConstantes.DIRETORIO_DADOS)
OSUtil.criar_diretorio(diretorio=ParametrosConstantes.DIRETORIO_LOGS)

log = LoggerUtil.configurar_logger(arquivo_log=ParametrosConstantes.CAMINHO_ARQUIVO_LOG_CARGA_INICIAL)

In [ ]:
log.info(msg="Iniciando o processamento da carga inicial.")

conexao_bd = BancoDadosUtil.estabelecer_conexao_banco()
carga_inicial_component = CargaInicialComponent()
carga_inicial_service = CargaInicialService()
ibge_client_service = IBGEClientService()
municipio_service = MunicipioService()
unidade_federativa_service = UnidadeFederativaService()

In [ ]:
log.info(msg="Inicializando o banco de dados.")

carga_inicial_service.dropar_tabelas_banco(conexao_bd)
carga_inicial_service.criar_tabelas_banco(conexao_bd)
carga_inicial_service.popular_tabelas_iniciais(conexao_bd)

In [ ]:
log.info(msg="Obtendo os shapefiles das unidades federativas e dos municípios.")

ibge_client_service.obter_shapefile_unidades_federativas()
ibge_client_service.obter_shapefile_municipios()

In [ ]:
log.info(msg="Montando os dados iniciais para a carga.")

gdf_unidade_federativa, gdf_municipio = carga_inicial_component.montar_carga_inicial()

In [ ]:
log.info(msg="Persistindo os dados na base.")

unidade_federativa_service.salvar_geodataframe(gdf=gdf_unidade_federativa, conexao_bd=conexao_bd)
municipio_service.salvar_geodataframe(gdf=gdf_municipio, conexao_bd=conexao_bd)

In [ ]:
log.info(msg="Encerrando a conexão com o banco de dados.")

BancoDadosUtil.encerrar_conexao_banco(conexao_bd)